# CNN Model

In [ ]:
class CNN(nn.Module):

    def __init__(self, act, pooling):
        super(CNN, self).__init__()
        self.act = act
        self.pooling = pooling
        
        self.conv1 = nn.Conv2d(in_channels = 3,
                               out_channels = 64,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)
        self.conv2 = nn.Conv2d(in_channels = 64,
                               out_channels = 256,
                               kernel_size = 5,
                               stride = 1,
                               padding = 2)
        
        if self.act == 'relu':
            self.act = nn.ReLU()
        elif self.act == 'tanh':
            self.act == nn.Tanh()
        elif self.act == 'sigmoid':
            self.act = nn.Sigmoid()
        else:
            raise ValueError('no valid activation function selected!')
       
        if self.pooling == 'max_pooling': 
            self.pooling = nn.MaxPool2d(kernel_size = 2, stride = 2)
        elif self.pooling == 'avg_pooling':
            self.pooling = AvgPool2d(kernel_size = 2, stride=2)
        else: 
            raise ValueError('Not a valid pooling code')
        
        self.fc = nn.Linear(65536, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act(x)
        x = self.conv2(x)
        x = self.act(x)
        x = self.pooling(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Experiment Code

In [ ]:
def experiment(partition, args):
    
    net = CNN(act = args.act,
              pooling = args.pooling)
    net.cuda()

    criterion = nn.CrossEntropyLoss()
    if args.optim == 'SGD':
        optimizer = optim.SGD(net.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(net.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(net.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')
    
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
        
    for epoch in range(args.epoch):  # loop over the dataset multiple times
        ts = time.time()
        net, train_loss, train_acc = train(net, partition, optimizer, criterion, args)
        val_loss, val_acc = validate(net, partition, criterion, args)
        te = time.time()
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        
        print('Epoch {}, Acc(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.2f}/{:2.2f}. Took {:2.2f} sec'.format(epoch, train_acc, val_acc, train_loss, val_loss, te-ts))
        
    test_acc = test(net, partition, args)    
    
    result = {}
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    result['train_accs'] = train_accs
    result['val_accs'] = val_accs
    result['train_acc'] = train_acc
    result['val_acc'] = val_acc
    result['test_acc'] = test_acc
    return vars(args), result

# Experiment

In [ ]:
# ====== Random Seed Initialization ====== #
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.exp_name = "exp1_basic_CNN_model_max_pooling"

# ====== Model ====== #
args.act = 'relu' 
args.pooling = 'max_pooling'  #'max_pooling' #'avg_pooling'

# ====== Regularization ======= #
args.l2 = 0.00001

# ====== Optimizer & Training ====== #
args.optim = 'Adam' #'RMSprop' #SGD, RMSprop, ADAM...
args.lr = 0.0001
args.epoch = 10

args.train_batch_size = 512
args.test_batch_size = 1024
                
setting, result = experiment(partition, deepcopy(args))
save_exp_result(setting, result)

#### Epoch 0, Acc(train/val): 38.76/47.00, Loss(train/val) 1.74/1.52. Took 34.73 sec
#### Epoch 1, Acc(train/val): 50.53/50.60, Loss(train/val) 1.41/1.39. Took 35.65 sec
#### Epoch 2, Acc(train/val): 55.60/54.82, Loss(train/val) 1.27/1.29. Took 34.93 sec
#### Epoch 3, Acc(train/val): 58.70/56.98, Loss(train/val) 1.18/1.22. Took 35.17 sec
#### Epoch 4, Acc(train/val): 61.12/59.23, Loss(train/val) 1.12/1.16. Took 35.27 sec
#### Epoch 5, Acc(train/val): 63.60/61.33, Loss(train/val) 1.06/1.12. Took 35.15 sec
#### Epoch 6, Acc(train/val): 65.04/62.58, Loss(train/val) 1.01/1.08. Took 35.06 sec
#### Epoch 7, Acc(train/val): 66.83/62.69, Loss(train/val) 0.97/1.07. Took 35.15 sec
#### Epoch 8, Acc(train/val): 68.28/63.05, Loss(train/val) 0.92/1.07. Took 35.17 sec
#### Epoch 9, Acc(train/val): 69.04/64.42, Loss(train/val) 0.90/1.03. Took 35.18 sec